Dieses Jupyter Notebook dient dem Analysieren und Visualisieren der angewandten Semantik von Bundestagsparteien im Plenarsaal.
Seine Funktion ist es alle gesprochenen Wörter herauszufiltern, die ausschließlich von einer Partei und nicht den anderen Parteien verwendet wurden, außerdem noch die von zwei Parteien gemeinsam und wiederum nicht von den anderen Parteien verwendet wurden. Das Ziel ist es, Parteien aufgrund ihres Jargons und semantischer Charakteristika zu unterscheiden, Zusammenhänge zu erkennen und eventuell einem politischen Spektrum zuordnen zu können.
Als Datensatz dienen die digitalisierten Reden in den Plenarprotokollen des deutschen Bundestages der 19ten Wahlperiode[1]. Mit den analysierten Materialien lassen sich Listen ausgeben und Datenvisualisierungen in Form von Wortgraphen erzeugen. Betrachtet werden die 6 Parteien, die über die 5 Prozent Klausel in den deutschen Bundestag gewählt wurden und ihn maßgeblich gestalten. Außen vor bleiben fraktionslose Kandidaten oder gesonderte Abgeordnete aus Bremen die ebenfalls im Datensatz vorkamen.

## Module und Bibliotheken einbinden

Ausgewählte Module und Bibliotheken werden bereitgestellt, die in den später konzipierten Methoden aufgerufen werden können. Als Softwareverzeichnis wird zu jeder Beschreibung zusätzlich ein Link eingefügt.
Zuerst wird das json-Modul importiert um json-Dateien (JavaScript Object Notations; z.B. die Bundestagsredenliste) nutzen zu können.
 https://www.json.org/json-en.html [2]
Die Matplotlib ist ein Visualisierungsmodul. Zu ihr gehört auch das Pyplot-Modul, welches herangezogen wird um Darstellung ähnlich der bekannten Matlab-Platform zu ermöglichen. Als Pseudonym wird dazu fortlaufend alternativ plt verwenden.  https://matplotlib.org/ [3]; https://matplotlib.org/3.5.1/tutorials/introductory/pyplot.html [4]
Die Wordcloud bildet die Häufigkeit vom Vorkommen von Wörtern mittels einer Grafik ab.
 https://python-graph-gallery.com/wordcloud/ [5]
PIL (Python Imaging Library) ist eine Bibliothek zum Arbeiten mit verschiedenen Bilddateiformaten.
 https://note.nkmk.me/en/python-pillow-basic/ [6]
Um später die Wordcloud zu erstellen fügen wir ebenfalls die NumPy-Bibliothek hinzu.
 https://numpy.org/ [7]
Hinzukommt Pythons Objektzähler, der Counter, zum Zählen der Wörter.
 https://www.geeksforgeeks.org/python-collections-module/ [8]
Als letzter Punkt wird, um die (sozialen) Netzwerke als Graph abbilden zu können, die Python-Bibliothek Networkx importiert.
 https://networkx.org/documentation/stable/index.html [9]

In [1]:
import json
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image
import numpy as np
from collections import Counter
import networkx as nx

## Importieren der lemmatisierten Reden

Alle Reden der Sitzungen vom deutschen Bundestag der 19ten Wahlperiode sind in Plenarprotokollen auf der Website des deutschen Bundestages in xml- und json-Dateien beziehbar. Zum schnelleren Bearbeiten dieser großen Menge an Daten (>24000 Reden), bietet sich für das Jupyter Notebook das weit verbreitete und leicht verarbeitbare json-Format an. Zur erheblichen Reduzierung der Rechenzeiten, wurden in einem Vorverarbeitungsschritt alle Reden in einer json-Datei zusammengefügt und ihre transkribierten Textteile lemmatisiert, d.h. die Wörter wurden, um Mehrfachauftreten von gleichen oder gleichmeinenden Wörtern zu verhindern, auf ihre Grundform zurückgeführt. So haben sie nun z.B. keine unterschiedliche Bedeutung mehr aufgrund verschiedener Satzzeichen an den Wörtern. Im Prinzip handelt es sich beim Inhalt der json-Datei um Listen der Lemmata der Wörter aus den Reden ohne Satzregeln, eine Art universale Rohform, aber semantisch gleichbleibend.
Die Funktion laden_alleReden soll die vorbereitete json-Datei nun laden und gibt eine Kopie der dieser Datei wieder, damit die Ursprungsdatei nicht bei der Bearbeitung überschrieben wird.
Durch Aufruf des Kürzels fp öffnet und liest die Funktion das über den Pfad angegebene Verzeichnis. Dadurch wird der Inhalt des Verzeichnisses geladen, in data gespeichert und eine Kopie für das Notebook erstellt.

In [2]:
def laden_alleReden():
    
    with open('../../data/speeches_20_with_comments.jsonl', 'r', encoding = 'utf8') as fp:
        data = list(fp)
    speeches = []
    for line in data:
        speeches.append(json.loads(line))
    
    with open('../../data/speeches_19_preprocessed.json', 'r') as fp:
        data = json.load(fp)
    
    return data.copy()

Die Funktion laden_alleReden wird nun mit alleReden aufgerufen und packt den Return in die Liste alleReden.

In [3]:
alleReden = laden_alleReden()

Die Struktur der in der json-Datei gebündelten Reden ist am folgenden Auszug ersichtlich. Eine gute Übersicht verleiht sogleich die erstgetätigte Rede mit dem Index 0. Zuerst sind alle Attribute gelistet, wie Datum der Rede, Name des Redners, Parteizugehörigkeit usw.. Darunter folgen der transkribierte Text der Rede, der gleiche Text in lemmatisierter Version, mit dem das Jupyter Notebook im Anschluss arbeitet und zuletzt stehen stellvertretend die grammatikalischen Begriffe an Stelle der Wörter und Satzzeichen.

In [4]:
#print(alleReden[0])

## Reden nach Parteien sortieren und zu Parteitexten zusammenfügen

Jeder Parteiname wird als ein Key deklariert und erhält ein Array, in dem man die Reden die zu dieser Partei gehören ablegen kann. Mittels der Attribute Redner und parteizugehörigkeit, über die jede Rede verfügt, werden alle Reden auf die jeweils zugehörige Partei aufgeteilt und dort, zu einer Parteiliste der Parteizugehörigen lemmatisierten Reden ergänzt, solange bis sämtliche Reden verteilt wurden. 
Protokollseitig wurden die Parteinamen nicht immer identisch notiert, weshalb abweichende Namensvariationen zusätzlich den üblichen Parteibezeichnungen zugewiesen werden müssen.

In [5]:
reden_gefiltert = {'CDU_CSU':[],
                   'SPD':[],
                   'AfD':[],
                   'FDP':[],
                   'BÜNDNIS 90_DIE GRÜNEN':[],
                   'DIE LINKE':[],
                   'fraktionslos':[],
                   'Bremen':[]
                  }

for rede in alleReden:
    rede['party']=rede['party'].replace(u'\xa0', u' ')
    rede['party']=rede['party'].replace(u'/', u'_')
    if rede['party']=='Bündnis 90_Die Grünen':
        rede['party']='BÜNDNIS 90_DIE GRÜNEN'
    if rede['party']=='Fraktionslos':
        rede['party']='fraktionslos'
    
    reden_gefiltert[ rede['party'] ].extend(rede['text_lem'])

KeyError: 'text_lem'

## Erstellen von unique words (uw) von Einzelparteien und Parteipaaren

Mit dieser Funktion sollen jene Wörter gefunden werden, die ausschließlich von einer Partei stammen und somit ihr Merkmal sind. Diese zuweisbaren Wörter werden im Folgenden "unique words" genannt. Das Herausfiltern dieser charakteristischen Wörter geschieht mit Hilfe einfacher Mengenoperationen die im Folgenden an Beispielen erklärt und verbildlicht werden sollen [10].
Die Funktion verwendet die Parteinamen in Form der erstellten Keys und bildet zwei Wörterbücher in Form von Sets.
Das eine enthält die Wörter der lemmatisierten Reden der zu untersuchenden Partei(Key).
Das andere vereinigt die Wörter der restlichen Parteiwörterbücher(restliche Keys) durch Anwendung
des union-Operators "|".
Die unique words der zu untersuchenden Partei sind das Resultat der Differenz der Wörter aus der Menge der eigenen Parteireden minus der Wörter aus der Menge der Parteireden der vereinigten restlichen Parteien.
Am Ende werden die unique words der Partei in Form einer Liste ausgegeben.

union-Operation "|"

x1={'foo','bar','baz'}
x2={'baz','qux','quux'}

x1 | x2
{'foo','qux','quux','baz','bar'}

<img src="imagedata/union.jpg" style="width:329px;height:224px"/>

difference-Operation "-"

x1={'foo','bar','baz'}
x2={'baz','qux','quux'}

x1 - x2
{'foo','qux','quux','baz','bar'}

<img src="imagedata/difference.jpg" style="width:329px;height:224px"/>

In [ ]:
def return_unique_words(partylist,word_dict):

    andereworte = set()
    parteienworte = set()
    for key in word_dict.keys():
        if key not in partylist:
            andereworte = andereworte | set(word_dict[key])
        else:
            parteienworte = parteienworte | set(word_dict[key])
    
    

    unique_words = parteienworte - andereworte
    
    return unique_words

## Unique words nach Parteien in Form von Keys sortiert

In dem Set uw_all werden nun die unique words aller Parteien herausgefiltert und unter ihren Partei-Keys abgelegt, wie eine sortierte Tabelle. Somit wird ein Dictionary erschaffen, mit den einzigartigen Wörtern der Parteien, abrufbar unter ihrem Parteinamen.

In [ ]:
uw_all = {'SPD': return_unique_words('SPD',reden_gefiltert),
'FDP': return_unique_words('FDP',reden_gefiltert),
'CDU_CSU': return_unique_words('CDU_CSU',reden_gefiltert),
'DIE LINKE': return_unique_words('DIE LINKE',reden_gefiltert),
'BÜNDNIS 90_DIE GRÜNEN': return_unique_words('BÜNDNIS 90_DIE GRÜNEN',reden_gefiltert),
'AfD': return_unique_words('AfD',reden_gefiltert)
}

Die gesammten unique words einer Partei lassen sich jetzt mit Angabe des Partei-Keys anschauen.

In [ ]:
#print(uw_all['SPD'])

Ausgabe der Anzahl der unique words einer gewählten Partei.

In [ ]:
len(uw_all['SPD'])

## Speichern

Die Liste der unique words einer jeden Partei wird als json-Datei im Verzeichnis bereitgestellt.
Zuerst wird eine Leere Menge erschaffen, die Listen werden zurückgegeben, mit dem Inhalt, der bei uw_all unter dem Parteinamen als Key gespeichert ist.
Mittels der open-Funktion wird ermöglicht bei Eingabe des Kürzels unter eingetragenem Dateipfad eine json-Datei zu schreiben. In dieser json-Datei wird dann alles aus der Liste aus dem oberen Teil abgespeichert und im Verzeichnis abgelegt. Sort_keys=true ordnet das Geschriebene den Keys zu, während indent für ein gleichmäßiges und optisch vorteilhafteres Einrücken der Wörter im Text sorgt.

In [ ]:
uw_all_lists = {}
for key in uw_all:
    uw_all_lists.update({ key : list(uw_all[key]) })

with open('results/unique_words_per_party.json', 'w') as fp:
    json.dump(uw_all_lists, fp, sort_keys=True, indent=4)

## Erstellen von shared unique words (suw) von Parteipaaren

Um weitere Charakteristika in der Semantik parteiübergeordneter politischer Blöcke zu untersuchen, werden die gemeinsamen unique words paarweiser Parteien erstellt. Also erst eine Schnittmenge der Wörter die zwei Parteien vereinen und dann mit einem gesammelten Wörterbuch der übrigen Parteien vergleichen. Somit lässt sich eventuell eine sprachliche Nähe und gemeinsamer häufiger Gebrauch bestimmter Wörter zwischen Parteien ausfindig machen.
Die unique words-Listen werden wie ein Schachbrett in Reihe und Spalte aufgelistet und die Zweierkombinationen erstellt. Diesmal wird die reden_gefiltert-Funktion für das gewünschte Parteienpaar aufgerufen, alle ihre Wörter zusammengelegt und die unique words Listen der übrigen 4 Parteien wieder gebündelt subtrahiert. Da in diesem Fall die gemeinsamen unqiue words der beiden Parteien gesucht werden, werden nochmal die unique words die jeweils nur in einer Partei vorkommen davon abgezogen.
Zum Kombinieren der Parteien wird dazu in suw in aufsteigender Zählweise mit der Partei in der i. Spalte begonnen und für diese erstmal alle Kombinationsmöglichkeiten mit den Parteien in den j Reihen erstellt, wobei mit j einer Spaltenzahl höher beginnend, damit keine Schnittmenge von i mit sich selbst entsteht, bzw. die Diagonale der Tabelle ausgelassen wird. Da die Partei-Keys aus der uw_all Liste keinen Zahlen in Rangfolgen entsprechen, wird mit enumerate nach Art der for-Schleife durch das Dictionary iteriert und dazu die Partei-Keys abgezählt, so dass sie unter entsprechender Spalten- und Reihennummer wieder auftauchen.
Insgesamt erscheinen 15 verschiedene Kombinationsmöglichkeiten oder Wordclouds der shared unique words.
Zusätzlich wird durch die Schleife durch suw_words die Anzahl der shared unique words einer jeden Partei mit den jeweils anderen Parteien wiedergegeben.

In [ ]:
rows, cols = (len(uw_all), len(uw_all))
suw = [[0 for i in range(cols)] for j in range(rows)]
suw_words = [[0 for i in range(cols)] for j in range(rows)]
for ix,p1 in enumerate(uw_all.keys()):
    for jx,p2 in enumerate(uw_all.keys()):
        if jx > ix:
            print(p1,p2)
            tmp = return_unique_words([p1,p2],reden_gefiltert) - return_unique_words(p1,reden_gefiltert) - return_unique_words(p2,reden_gefiltert)
            suw_words[ix][jx] = tmp
            suw[ix][jx] = len(tmp)
            suw[jx][ix] = suw[ix][jx]

print(suw)

#0 SPD, 1 FDP, 2 CDU/CSU, 3 Linke, 4 Grüne, 5 AfD

Die unique words eines bestimmten Parteienpaares lassen sich jetzt mit Angabe der Parteitabellenplatzzahlen öffnen.

In [ ]:
#print(suw_words[0][1])

#print(uw_all.keys())
#0 SPD, 1 FDP, 2 CDU/CSU, 3 Linke, 4 Grüne, 5 AfD

## Wordcloud einer einzelnen Partei

Nun wird aus den Listen zur Veranschaulichung eine Wordcloud erstellt. Bei einer Wordcloud entspricht die relative Schriftgröße der dargestellten Wörter ihren prozentualen Häufigkeiten ihres Vorkommens in der Liste.
Es wird der Partei-Key der zu visualisierenden Partei verwendet.
In der Counter Funktion werden die Wörter der unique word Liste der Partei iterativ durchlaufen und bei jedem geschaut, wie oft es im ungekürzten Parteitext vorkommt, der all ihre Reden enthält um die Häufigkeit des verwendeten Wortes festzustellen.
Die imagemask bzw. Formstruktur der Wordcloud wird durch ein selbstgewähltes Bild (Kreis) festgelegt, wobei ein png-Dateiformat notwendig ist.
Weiter lassen sich als Grundeinstellung ein paar Parameter festlegen, wie Detailgrad, Farben und die Quelle für die anzuzeigenden Wörter in Form der Counterausgabe. Der Farbhintergrund wird als transparent festgelegt, wofür das 4kanalige Farbmodell RGBA (RotGrünBlauAlpha) bei background_color und mode = "RGBA" eingestellt werden müssen.
Zum Schluss wird die Abbildung der Wordcloud im Notebook generiert. Die Maske dafür war das Kreis-png.
Die Kreisform der Wordcloud ergibt sich, da sie nur im Areal der schwarzen Pixel rasterisiert(imshow) wird.

In [ ]:
party = 'AfD'
#party = 'CDU_CSU'
#party = 'SPD'
#party = 'FDP'
#party = 'BÜNDNIS 90_DIE GRÜNEN'
#party = 'DIE LINKE'

uw_counter = [wort for wort in reden_gefiltert[party] if wort in uw_all[party]]
counts = Counter(uw_counter)

mask = np.array(Image.open("circle.png"))

# parameters:

wordcloud = WordCloud(width=3200, height=3200, mode = "RGBA", background_color='rgba(255, 255, 255, 0)', max_words=500, mask=mask, contour_color='#000000', contour_width=0, colormap='rainbow').generate_from_frequencies(counts)



# create image as circle:

fig = plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# presentate:

plt.show()

## Speichern als Rasterbild

Genügt die Darstellung den Ansprüchen, ist sie als "normales Bild", als Rasterbild im png Format, auf dem Dateipfad mit dem jeweiligen Dateinamen am Ende abspeicherbar.

In [ ]:
wordcloud.to_file(f"results/wordcloud_{party}.png")

## Speichern als SVG

Für die Zwecke einer qualitätsverlustfreien Vergrößerung der Wordcloud zum pixelfreien Heranzoomen, oder
z.B. zur Anbringung auf großflächigen Drucken, lässt sich die Darstellung auch als Vektorgrafik in Form einer svg-Datei abspeichern. 
w+ ermöglicht Dateien sowohl zu lesen als auch zu schreiben, encoding="utf-8" decodiert die Datei aus dem Jupyter Notebook Skript im Pythoncode.
Kleinere Mängel können beim Resultat dieser Prozedur jedoch entstehen, da abhängig von den Leseprogrammen die für die Grafik im Anschluss verwendet werden, sich die Anordnung der Wörter zueinander verschieben kann, oder auch Sonderzeichen aus Python z.B. nicht erkannt und ersetzt werden.

In [ ]:
party='BÜNDNIS 90_DIE GRÜNEN'
wordcloud_svg = wordcloud.to_svg(embed_font=True)
f = open(f"results/wordcloud_{party}.svg","w+", encoding="utf-8")
f.write(wordcloud_svg )
f.close()

## Beispielwordclouds der 6 Parteien

Wordclouds der 6 Parteien chronologisch sortiert (von li. nach re., von o. nach u.) nach Anzahl unique words:
CDU/CSU (38312), AfD (24337), SPD (23379), Die Linke (12417), FDP (12035), Bündnis 90/Die Grünen (11960)

<img src="imagedata/parties1.jpg" style="width:787px;height:382px"/>

<img src="imagedata/parties2.jpg" style="width:787px;height:382px"/>

<img src="imagedata/parties3.jpg" style="width:787px;height:382px"/>

## Wordcloud für Parteipaar

Die Funktion erstellt eine Wordcloud aller selbst gewählten Parteipaare.

In [ ]:
#0 SPD, 1 FDP, 2 CDU/CSU, 3 Linke, 4 Grüne, 5 AfD

uw_counter = [wort for wort in reden_gefiltert['SPD'] + reden_gefiltert['FDP'] if wort in suw_words[0][1]]

counts = Counter(uw_counter)

mask = np.array(Image.open("circle.png"))

# parameters:

wordcloud = WordCloud(background_color='white', max_words=500, mask=mask, contour_color='#000000', contour_width=3, colormap='rainbow').generate_from_frequencies(counts)



# create image as circle

fig = plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# presentate:

plt.show()

## Speichern als Rasterbild

In [ ]:
wordcloud.to_file(f"results/wordcloud.png")

## Speichern als SVG

Funktion zum Abspeichern der Wordcloud der Parteipaare als Vektorgrafik.

In [ ]:
wordcloud_svg = wordcloud.to_svg(embed_font=True)
f = open("results/wordcloud.svg","w+", encoding="utf-8")
f.write(wordcloud_svg )
f.close()

## Graph erstellen

Als besondere Darstellungsmethode von (sozialen) Netzwerken, kann ein Netzwerk ausgegeben werden, das sich mit der Software Gephi [11] als Graph visualisieren lässt. Dafür wird im Jupyter Notebook ein gewichtetes Knoten- und Kantennetzwerk aus den unique words-Listen erzeugt. Als erstes werden die Knoten erzeugt und mit den Attributen Identifikationsnummer, Parteinamen und der Größe der parteizugehörigen unique words-Liste ausgestattet und befüllt. Das geschieht durch enumerate-Schleifen über die Keys.
Append fügt der Liste "nodes" jeden Knoten als Item hinzu.

In [ ]:
nodes = []
for px,partei in enumerate(uw_all.keys()):
    node = {
        'id' : px,
        'name' : partei,
        'nuw' : len(uw_all[partei])
    }
    nodes.append(node)
    
nodes

Ein ungerichteter Graph wird erstellt, als edgestrength_graph deklariert und bekommt die vorher angefertigten nodes als Knoten und noch zu berechnende Links als Kanten für den Graphen.
Für die Links bzw. Kanten, werden alle Knoten miteinander verglichen. Es gibt über jede Kante zwei inzidente Knoten, einer als Quelle einer als Ziel, wobei die Kantenstärke zwischen den Knoten auf Gewichten basiert, die sich aus den geteilten unique words (suw) errechnen.
Je mehr geteilte Wörter es gibt, desto stärker ist das Gewicht zwischen den Parteien.
Stückweise wird jede fertig-berechnete Kante wird dem Graphen hinzugefügt.

In [ ]:

graph = {
    'directed': False,
    'graph': 'edgestrength_graph',
    'links': [],
    'nodes': nodes
}    

for ix,nodeI in enumerate(graph['nodes']):
    for jx,nodeJ in enumerate(graph['nodes']):
        if jx > ix:
            
            weight = suw[ix][jx]
            source = nodeI['id']
            target = nodeJ['id']
            link_dict = {
                    'source':source,
                    'target':target,
                    'weight':weight       
                }
            graph['links'].append(link_dict)
    
print(graph)

Jeder Knoten wird an Gephi übergeben, bis alle Knoten fertig sind. Dann folgen die Kanten.
Eine xml-Datei wird im selben Ordner wie das Notebook abgespeichert.

In [ ]:
graphforgephi = nx.Graph()
for node in graph['nodes']:
    graphforgephi.add_node(node['id'],name = node['name'],nuw = node['nuw']);
    
    
print('nodes done')
for link in (graph['links']):   
    graphforgephi.add_edge(link['source'],link['target'],weight = link['weight'])
print('links done')   
nx.write_gexf(graphforgephi, "results/graphforgephi.gexf")
print('save done')

<img src="imagedata/gephi_graphs_suw.jpg" style="width:1279.58px;height:382px"/>

## Auswertung der Ergebnisse und Erfahrungen aus der Arbeit mit dem Jupyter Notebook

Das Juypter Notebook in seiner abgeschlossenen Version besticht durch hohe Funktionalität und liefert einen geeigneten Output der einem einen Einblick in die Semantik der Parteien verschafft. Dem liegt ein größerer Prozess aus Vorüberlegungen, Systemanalyse, stufenweiser Bearbeitung, Reflexion, Codeanpassungen und Zielabgleichen zugrunde. Einerseits gewann der Code über die Bearbeitungszeit deutlich an Übersichtlichkeit, Abgestimmtheit und insbesondere an gesteigerter Effizienz. Parallel war es immer wieder nötig das Extrahieren der unique words aus den Textkörpern der Reden der Abgeordneten der Parteien mittels neuer methodischer Ansätze zu versuchen. Interessant dabei ist, dass das anvisierte Ergebnis und einige Etappenziele recht früh definiert und ohne große Beschränkung realisiert werden konnten.

Grundsätzlich lässt sich im Résumé ein Wesen der Parteien aus den unique words-Listen erkennen. Während die Regierungsparteien CDU/CSU und SPD in hoher Frequenz lösungsgerichtete Perspektiven in ihren unique words kommunizieren (Klimainnovation, Lösung, Aufholpaket, Zukunftsbündnis, Minderheitenpolitik), sticht z.B. die stärkste Oppositionspartei AfD mit weitaus kritischeren Wortkreationen zu Hauf heraus (Ökosozialismus, Klimasekte, Coronahysterie, linksgrünen, Merkel-Regierung, Pack). Eine Ähnlichkeit mit medial etablierten politischen Narrativen ist hierin wiederzuerkennen und durch das Jupyter Notebook in ihrer Intensität quantifizierbar. Ebenso sind die politischen Rollenbilder der debattierenden Parteien  (Regierung-Opposition) deutbar. Dabei sei jedoch zu beachten, dass mit Blick auf die methodengebundene Häufigkeitsanalyse der unique words nur ein Ausschnitt der Reden und lemmatisierte Wörter ohne ihre einrahmenden sinnerweiternden Texte wiedergegeben werden konnte. Wurden z.B. Wörter auch in ironischer Absicht verwendet? Die jeweiligen parteiinneren Gruppendiversitäten sind hierbei kaum auszumachen. Eine kleine Ausnahme stellt die SPD mit plattdeutschen Wortanteilen (hebben, Spraak). Im Großen und Ganzen lässt sich aber auf Grund der großen Häufigkeiten ähnlicher Wörter eine Tendenz abschätzen.

## Literaturverzeichnis

[1] Deutscher Bundestag Information und Kommunikation Referat Online-Dienste (2022, 11. April). Open Data. https://www.bundestag.de/services/opendata

[2] Website (2022, 11. April). Introducing JSON. https://www.json.org/json-en.html

[3], [4] J. D. Hunter (2022, 11. April).  "Matplotlib: A 2D Graphics Environment", Computing in Science & Engineering, vol. 9, no. 3, pp. 90-95, 2007. https://matplotlib.org/. https://matplotlib.org/3.5.1/tutorials/introductory/pyplot.html

[5] Yan Holtz (2022, 11. April). Wordcloud. https://python-graph-gallery.com/wordcloud/

[6] nkmk (2019, 14. Mai). How to use Pillow (PIL: Python Imaging Library). https://note.nkmk.me/en/python-pillow-basic/

[7] NumFOCUS (2022, 11. April). https://numpy.org/

[8] GeeksforGeeks (2021, 19. November). Python Collections Module. https://www.geeksforgeeks.org/python-collections-module/

[9] Networkx Developers (2022, 9. April). https://networkx.org/documentation/stable/index.html

[10] John Sturtz (2022, 11. April). Sets on Python. https://realpython.com/python-sets/

[11] Gephi (2022, 11. April). Learn how to use Gephi. https://gephi.org/users/

Letzter Zugriff auf alle Websites des Verzeichnisses, Datum: 2022, 11. April.